In [93]:
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

BASE_WORKS = "https://api.penguinrandomhouse.com/resources/v2/title/domains/PRH.US/works"
BASE_WORK_TITLES = "https://api.penguinrandomhouse.com/resources/v2/title/domains/PRH.US/works"
BASE_TITLES = "https://api.penguinrandomhouse.com/resources/v2/title/domains/PRH.US/titles"


In [94]:
API_KEY = "fh5hj47dynk4nvx4s9ewufj4"

In [95]:
DOMAIN = "PRH.US"

BASE_WORKS = "https://api.penguinrandomhouse.com/resources/v2/title/domains/PRH.US/works"
BASE_TITLE = "https://api.penguinrandomhouse.com/resources/v2/title/domains/PRH.US/titles"

In [96]:
def get_works_page(start, rows, on_sale_from):
    params = {
        "api_key": API_KEY,
        "rows": rows,
        "start": start,
        "preferLanguage": "E",
        "ageRangeMax": 18,
        "ageRangeMin": 7,
        "showPublishedBooks": "true",
        "showComingSoon": "false",
        "showNewReleases": "true",
        "showFlapCopy": "true",
        "suppressLinks": "true",
        "suppressRecordCount": "true",
        "onSaleFrom": on_sale_from,  # mm/dd/yyyy
    }

    r = requests.get(BASE_WORKS, params=params, timeout=15)
    r.raise_for_status()
    data = r.json()

    # IMPORTANT: works list is here
    return data["data"]["works"]

In [97]:
def get_work_titles(workId):
    url = f"{BASE_WORK_TITLES}/{workId}/titles"
    params = {"api_key": API_KEY}

    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
    data = r.json()

    # The correct structure for your API:
    # { "titles": [ {...}, {...}, ... ] }
    titles = data.get("titles", [])

    # Guarantee titles is a list of dicts
    if isinstance(titles, list):
        titles = [t for t in titles if isinstance(t, dict)]
    else:
        titles = []

    return titles

In [98]:
def fetch_title(isbn):
    url = f"{BASE_TITLES}/{isbn}"
    params = {"api_key": API_KEY}

    r = requests.get(url, params=params, timeout=15)
    if r.status_code == 404:
        return None
    r.raise_for_status()
    return r.json()

def extract_description(title_data):
    for field in ["flapCopy", "description", "longDescription", "shortDescription"]:
        if field in title_data and title_data[field]:
            return title_data[field]
    return None

In [99]:
def collect_corpus(max_titles=1000):
    one_year_ago = (datetime.now() - timedelta(days=365)).strftime("%m/%d/%Y")

    isbns = []
    texts = []

    start = 0
    rows = 20

    while len(isbns) < max_titles:

        works = get_works_page(start=start, rows=rows, on_sale_from=one_year_ago)
        if not works:
            break

        for w in works:
            workId = w.get("workId")
            if not workId:
                continue

            # Fetch titles for this work
            title_records = get_work_titles(workId)

            title_records = get_work_titles(workId)

            for t in title_records:
                isbn = t.get("isbn")
                if not isbn:
                    continue

                # Fetch description
                title_data = fetch_title(isbn)
                if not title_data:
                    continue

                desc = extract_description(title_data)
                if not desc:
                    continue

                isbns.append(isbn)
                texts.append(desc)

                if len(isbns) >= max_titles:
                    break

            if len(isbns) >= max_titles:
                break

        start += rows
        time.sleep(0.2)

    return isbns, texts


In [100]:
sample_titles = get_work_titles(262934)  # using the first workId you showed earlier
print("Type:", type(sample_titles))
print(sample_titles)

Type: <class 'list'>
[]


In [102]:
isbns, texts = collect_corpus(max_titles=50)

KeyboardInterrupt: 

In [ ]:
vec = TfidfVectorizer(stop_words="english", max_features=5000)
X = vec.fit_transform(texts)

print("Corpus size:", len(texts))
print("TF-IDF shape:", X.shape)

NameError: name 'texts' is not defined

In [ ]:
sample = get_works_page(start=0, rows=5, on_sale_from="10/01/2025")
print(type(sample))
print(sample.keys())
print(sample["data"][:200] if isinstance(sample["data"], str) else sample["data"])


In [ ]:
vec = TfidfVectorizer(stop_words="english", max_features=5000)
X = vec.fit_transform(texts)


NameError: name 'texts' is not defined

In [ ]:
pd.DataFrame({"isbn": isbns, "text": texts}).to_csv("prh_corpus.csv", index=False)
sparse.save_npz("prh_tfidf.npz", X)

print("Corpus size:", len(isbns))
print("TF-IDF shape:", X.shape)

NameError: name 'isbns' is not defined